# Seminario: Organizar sesiones de doblaje
*Adrien Felipe*

## Analisis del enunciado
### Días de grabación
Hay 30 tomas en total y no se pueden superar 6 tomas por día,
por lo cual el caso óptimo no puede ser inferior a **5 días de grabación**.
### Precio por actor
Al cobrar todos los actores lo mismo por día independientemente de número de tomas, podemos **ignorar el precio por actor**.   

## Librerias

In [53]:
import pandas as pd
import numpy as np

## Preparación datos
Paso a un CSV los datos para tenerlos disponibles con pandas.

In [7]:
filepath = '../res/data.csv'
dataFrame = pd.read_csv(filepath, index_col=0)
# Pretty print only first 5 items.
display(dataFrame.head())

,1,2,3,4,5,6,7,8,9,10
Toma,,,,,,,,,,
1,1,1,1,1,1,0,0,0,0,0
2,0,0,1,1,1,0,0,0,0,0
3,0,1,0,0,1,0,1,0,0,0
4,1,1,0,0,0,0,1,1,0,0
5,0,1,0,1,0,0,0,1,0,0


## Analisis de los datos

En la tabla a continuación vamos a describir el total de tomas por actor, y su mínimo de días de grabación por actor. 

In [9]:
# Maximo de tomas por día.
max_takes = 6
# Columnas del DataFrame.
takes_col = 'takes'
actor_col = 'actor'
days_col = 'min days'

actors_data = pd.DataFrame(dataFrame.sum())
actors_data.columns = [takes_col]
actors_data.index.names = [actor_col]
# Calculamos el minimo de dias necesarios por actor, un dia parcial siendo un dia completo.
actors_data[days_col] = np.ceil(actors_data[takes_col] / max_takes)
display(actors_data)

# Sumamos los minimos individuales para conocer el minimo total.
print('Mínimo de gastos por servicios de actores: %i' % actors_data[days_col].sum())

,takes,min days
actor,,
1,22,4.0
2,14,3.0
3,13,3.0
4,15,3.0
5,11,2.0
6,8,2.0
7,3,1.0
8,4,1.0
9,2,1.0


Mínimo de gastos por servicios de actores: 21


Vemos entonces que el gasto mínimo inmejorable es de **21 servicios** de actores.

## Resolución por fuerza bruta

La complejidad de resolución por fuerza bruta, corresponde en generar todas las combinaciones de tomas por sesión, teniendo en cuenta que el orden de la tomas por sesión es irrelevante, asi como el orden de las sesiones.
     
Si consideramos $n$ el total de tomas y $m$ las tomas por sesión, tenemos:     
Total de sesiones posibles: $\prod_{i=0}^{m-1} (n-i) = \frac{n!}{(n-m)!}$    
Y sesiones con las mismas tomas, pero con orden diferente = $m!$    
Con $n,m \in \mathbb{N}^2 $ y $1\leq m \leq n$     

Asi que al ignorar el orden de las tomas, tenemos:     
Total sesiones unicas = $\frac{n!}{(n-m)! \cdot m!}$    

La grabación nesecitará realizar $\frac{n}{m}$ sesiones en total ($m \neq 0$), y no se han de repetir las tomas en diferentes sesiones, por lo cual en cada sesión hemos de eliminar las tomas anteriores, obteniendo el total de combinaciones únicas de sesiones:       
       
$$ S = \prod_{i=0}^{ \frac{n}{m}-1} \frac{(n-i \cdot m)!}{(n-i \cdot m -m)! \cdot m!} $$

Si $m=1$,
$ S = \prod_{i=0}^{n-1} \frac{(n-i)!}{(n-i-1)!} = \prod_{i=0}^{n-1} (n-i) = n!$    

Si $m=n$,
$
S = \prod_{i=0}^{0} \frac{(n-i \cdot n)!}{(n-i \cdot n -n)! \cdot n!}
= \frac{n!}{(0)! \cdot n!} = 1
$

La complejidad de la resolución por fuerza bruta depende del valor de tomas por sesión de tal forma que:   

$$ O(n) \leq O(f) \leq O(n!) $$    

En el caso del enunciado, el número de combinaciones es:     
    
$$
\frac{30!}{(30-6)! \cdot 6!} *
\frac{24!}{(24 -6)! \cdot 6!} *
\frac{18!}{(18 -6)! \cdot 6!} *
\frac{12!}{(12 -6)! \cdot 6!} *
\frac{6!}{(6 -6)! \cdot 6!}
\approx 1,4 \cdot10^{18}
$$
      
      
            
**La resolución por fuerza bruta es claramente inviable**.


Vemos también que:
$$ S = \prod_{i=1}^{\frac{n}{m}} \frac{ (i \cdot m)! }{ ((i-1) \cdot m)! \cdot  m! } $$

# Resolución por ramificación y poda

## Estimación del coste inferior
Para estimar el coste inferior calculamos -por cada actor- cuantas sesiones mínimas son necesarias para cumplir con las tomas restantes de forma consecutiva. Y finalmente sumamos el mínimo de cada actor para obtener la estimación del coste inferior.    
Para ello sumamos todos los vectores de coste de tomas restantes, dividimos por el máximo de tomas por sesión, y redondeamos a la alta al ser el coste de una sesión es independiente de su número de tomas.

Definimos $n'$ el número de tomas restantes y $\vec{C}_{i}$ los vectores restantes de coste de toma.     

$$ \text{vector coste inferior}: \vec{C} = \frac{1}{m} \sum_{i=1}^{n'}  \vec{C}_{i} $$

$$ \text{coste inferior} = \lVert \text{techo} (\vec{C}) \lVert $$


## Tipos de poda
La idea es usar ramificación y poda con 3 tipos de poda:

### Poda de hijos de un mismo nodo
Tras generar los hijos de un nodo, nos podemos quedar con los hijos de menor estimación de coste inferior, y podar el resto.

### Poda de nodos con sesión completa
Tras completar una sesión -es decir que esta haya llegado al máximo de tomas por sesión- se pueden podar los nodos cuya suma del *coste actual* y del *coste restante estimado* sea mayor. Esto porqué los costes de la sesión no se fusionarán con los coste restantes estimados al estar la sesión completa.

### Poda de límite de nodos
Para evitar generar una lista demasiada importante de nodos a pesar de las podas anteriores, conviene aplicar una poda "dura" cuando se supere un número definido de nodos. En cuyo caso se ordenan los nodos de menor a mayor suma de *coste* y *estimación* y nos quedamos con los $x$ primeros difinidos por el límite.

In [13]:
def calculate_cost(solution: tuple, max_takes: int, costs: tuple) -> int:
    """Calculate the real cost of a single solution"""
    # Solution total cost.
    solution_cost = 0
    # Takes per session counter.
    takes_count = 0
    session_cost = None
    session_is_full = False

    for take in solution:
        takes_count += 1
        # Whether a session reached its max allowed takes.
        session_is_full = takes_count == max_takes
        # Use a numpy array to ease bitwise operations.
        take_cost = np.array(costs[take])
        # Use bitwise 'or' operation to join the takes' costs per session,
        # as an actor gets paid the same amount regardless of his takes by session.
        session_cost = take_cost if session_cost is None else session_cost | take_cost

        if session_is_full:
            # Total cost is the cost sum of all sessions.
            solution_cost += session_cost.sum()
            # Reset session cost and counters.
            session_cost = None
            takes_count = 0

    # Sum last session if it was not fully filled.
    if not session_is_full and session_cost is not None:
        solution_cost += session_cost.sum()

    return solution_cost


def estimate_lowest_cost(solution: tuple, max_takes: int, costs: tuple) -> int:
    """Calculate an estimation of the lowest cost from the remaining takes"""
    estimated_cost = None
    for take_id, take in enumerate(costs):
        if take_id not in solution:
            take_cost = np.array(take)
            estimated_cost = take_cost if estimated_cost is None else take_cost + estimated_cost

    # Estimate best case scenario cost, by summing the minimal needed sessions per actor in the remaining takes.
    return np.ceil(estimated_cost / max_takes).sum() if estimated_cost is not None else 0


def create_child_solutions(solution: tuple, costs: tuple, max_takes: int, explored_nodes=None) -> list:
    """From a node solution, creates its child solutions"""
    if explored_nodes is None:
        explored_nodes = []

    # Calculate takes number in the current session.
    session_takes_count = max_takes - (len(solution) + 1) % max_takes

    child_solutions = []
    for take_id in range(len(costs)):
        # Ignore already used takes.
        if take_id not in solution:
            local_solution = solution + (take_id,)
            # Sort takes in the current session to have unique solutions, regardless of its takes order.
            session_takes = tuple(sorted(local_solution[-session_takes_count:]))
            local_solution = local_solution[:-session_takes_count] + session_takes
            # Only add a child if it was not previously added.
            if local_solution not in explored_nodes:
                child_solutions.append(local_solution)
                # Keep track of generated solutions.
                explored_nodes.append(local_solution)

    return child_solutions


def branch_and_pound(costs: tuple, max_takes: int, nodes_limit: int = 500) -> tuple:
    """
    Iteratively branch and pound the node tree based on 3 types of pounding:
    - Node child pounding
    - Full session nodes pounding
    - Nodes limit pounding
    """
    best_solution = None
    best_solution_cost = None
    explored_nodes = []
    iteration = 0

    # Initialize the tree with an empty root node.
    nodes = [{
        's': (),
        'cost': 0,
        'depth': 0,
        'estimation': estimate_lowest_cost((), max_takes, costs),
    }]

    # Loop as long as there are nodes to loop over.
    while nodes:
        iteration += 1

        # Active nodes max and min depths.
        min_depth = min(nodes, key=lambda node: node['depth'])['depth']
        max_depth = max(nodes, key=lambda node: node['depth'])['depth']

        # Whether all nodes are at the same depth.
        if max_depth != 0 and min_depth == max_depth:
            # Whether it is a 'full session' depth.
            if min_depth % max_takes == 0:
                # As all nodes have full sessions, pound the ones with higher estimated cost.
                best_node = min(nodes, key=lambda node: node['cost'] + node['estimation'])
                best_cost = best_node['cost'] + best_node['estimation']
                nodes = [node for node in nodes if node['cost'] + node['estimation'] <= best_cost]

            # Hard pound if nodes list becomes too big. This might result in removing the best solution.
            elif len(nodes) > nodes_limit:
                # Sort nodes by current estimated cost.
                nodes = sorted(nodes, key=lambda node: node['cost'] + node['estimation'])
                nodes = nodes[:nodes_limit]

        # Extract first node of the list.
        # Node is expected to have the lowest depth as children are appended at the bottom of the list.
        selected_node = nodes.pop(0)

        # Branch selected node.
        child_nodes = [
            {
                's': solution,
                'depth': len(solution),
                'cost': calculate_cost(solution, max_takes, costs),
                'estimation': estimate_lowest_cost(solution, max_takes, costs),
            }
            for solution in create_child_solutions(selected_node['s'], costs, max_takes, explored_nodes)
        ]

        # Pound new branches with a higher estimated utopian cost.
        if child_nodes:
            best_cost = min(child_nodes, key=lambda node: node['estimation'])['estimation']
            child_nodes = [node for node in child_nodes if node['estimation'] == best_cost]
            nodes.extend(child_nodes)

        # Whether node is final and cannot have children.
        is_final_node = len(selected_node['s']) == len(costs)
        # Select node as best solution if its cost has improved.
        if is_final_node and (best_solution_cost is None or selected_node['cost'] < best_solution_cost):
            best_solution_cost = selected_node['cost']
            best_solution = selected_node['s']

    print('Best solution with cost %i in %i iterations:' % (best_cost, iteration))
    print(best_solution)

In [ ]:
%%time

# Lista de costes por toma
costs = tuple(dataFrame.itertuples(index=False, name=None))
# Tomas por sesión
max_takes = 6

branch_and_pound(costs, max_takes, 300)